### Skiier (1)

In [169]:
import numpy as np
import pandas as pd

In [170]:
# Model
states = np.arange( 0, 80, 10 ).astype( str )

P_normal = pd.DataFrame( np.matrix( [[0, 1, 0, 0, 0, 0, 0, 0],
                                     [0, 0, 1, 0, 0, 0, 0, 0],
                                     [0, 0, 0, 1, 0, 0, 0, 0],
                                     [0, 0, 0, 0, 1, 0, 0, 0],
                                     [0, 0, 0, 0, 0, 1, 0, 0],
                                     [0, 0, 0, 0, 0, 0, 1, 0],
                                     [0, 0, 0, 0, 0, 0, 0, 1],
                                     [0, 0, 0, 0, 0, 0, 0, 1]] ), index = states, columns = states )

P_speed = pd.DataFrame( np.matrix( [[0.1, 0, 0.9, 0, 0, 0, 0, 0],
                                    [0.1, 0, 0, 0.9, 0, 0, 0, 0],
                                    [0, 0.1, 0, 0, 0.9, 0, 0, 0],
                                    [0, 0, 0.1, 0, 0, 0.9, 0, 0],
                                    [0, 0, 0, 0.1, 0, 0, 0.9, 0],
                                    [0, 0, 0, 0, 0.1, 0, 0, 0.9],
                                    [0, 0, 0, 0, 0, 0.1, 0, 0.9],
                                    [0, 0, 0, 0, 0, 0, 0, 1]] ), index = states, columns = states )

R_s_a = pd.DataFrame( np.c_[[-1, -1, -1, -1, 0, -1, -1, 0], [-1.5, -1.5, -1.5, -1.5, -0.5, -1.5, -1.5, 0]],
                      index = states, columns = ['n', 's'] )

q_s_a_init = pd.DataFrame( np.c_[np.repeat( 0.0, len( states ) ), np.repeat( 0.0, len( states ) )], index = states,
                      columns = ['n', 's'] )


### Skiier (2)

In [171]:
# Policy
pi_speed = pd.DataFrame( np.c_[np.repeat( 0, len( states ) ), np.repeat( 1, len( states ) )], index = states,
                         columns = ['n', 's'] )

pi_50 = pd.DataFrame( np.c_[np.repeat( 0.5, len( states ) ), np.repeat( 0.5, len( states ) )], index = states,
                      columns = ['n', 's'] )


### Skiier (3)

In [172]:
def simul_path(pi, P_normal, P_speed, R_s_a):
    s_now = "0"
    history_i = [s_now]

    while s_now != "70":
        if np.random.uniform() < pi.loc[s_now, "n"]:
            a_now = "n"
            P = P_normal

        else:
            a_now = "s"
            P = P_speed

        r_now = str( R_s_a.loc[s_now, a_now] )
        s_next = states[np.argmin( P.loc[s_now].cumsum() < np.random.uniform() )]
        history_i.extend( [a_now, r_now, s_next] )
        s_now = s_next

    return history_i
sample_path=simul_path(pi=pi_speed,P_normal=P_normal,P_speed=P_speed,R_s_a=R_s_a)
print(sample_path)

['0', 's', '-1.5', '20', 's', '-1.5', '40', 's', '-0.5', '60', 's', '-1.5', '70']


### Skiier (4)

In [173]:
# simul_step()
def simul_step(pi, s_now, P_normal, P_speed, R_s_a):
    if np.random.uniform() < pi.loc[s_now, "n"]:
        a_now = "n"
        P = P_normal

    else:
        a_now = "s"
        P = P_speed

    r_now = R_s_a.loc[s_now, a_now]
    s_next = states[np.argmin( P.loc[s_now].cumsum() < np.random.uniform() )]

    if np.random.uniform() < pi.loc[s_next, "n"]:
        a_next = "n"
    else:
        a_next = "s"

    sarsa = [s_now, a_now, r_now, s_next, a_next]

    return sarsa


sample_step = simul_step( pi_speed, "0", P_normal, P_speed, R_s_a )
print( sample_step )


['0', 's', -1.5, '20', 's']


### Skiier (5)

In [174]:
def pol_eval_MC(sample_path, q_s_a, alpha):
    q_s_a_copy= q_s_a.copy()
    
    for j in range( 0,len( sample_path ) - 1, 3 ):
        s = sample_path[j]
        a = sample_path[j + 1]
        G = np.sum(np.array(sample_path[j + 2:len( sample_path )-1:3]).astype( float ) )

        q_s_a_copy.loc[s,a] += alpha * (G - q_s_a_copy.loc[s, a])

    return q_s_a_copy


q_s_a=pol_eval_MC( sample_path, q_s_a = q_s_a_init, alpha = 0.1 )
q_s_a

,n,s
0,0.0,-0.50
10,0.0,0.00
20,0.0,-0.35
30,0.0,0.00
40,0.0,-0.20
50,0.0,0.00
60,0.0,-0.15
70,0.0,0.00


### skiier (6)

In [175]:
def pol_eval_TD(sample_step, q_s_a, alpha):
    q_s_a_copy= q_s_a.copy()
    s = sample_step[0]
    a = sample_step[1]
    r = sample_step[2]
    s_next = sample_step[3]
    a_next = sample_step[4]

    q_s_a_copy.loc[s,a] +=alpha*(r+q_s_a_copy.loc[s_next, a_next]-q_s_a_copy.loc[s,a])

    return q_s_a_copy

q_s_a=pol_eval_TD(sample_step, q_s_a_init, alpha = 0.1)
q_s_a

,n,s
0,0.0,-0.15
10,0.0,0.00
20,0.0,0.00
30,0.0,0.00
40,0.0,0.00
50,0.0,0.00
60,0.0,0.00
70,0.0,0.00


In [213]:
def pol_imp(pi, q_s_a, epsilon): # epsilon = exploration_rate
    pi_copy =pi.copy()
    for i in range(pi.shape[0]):
        # exploitation
        if np.random.uniform() > epsilon:
            pi_copy.iloc[i] = 0
            pi_copy.iloc[i, np.argmax(q_s_a.iloc[i,])] = 1

            

        else:
            # exploration
            pi_copy.iloc[i] = 1/q_s_a.shape[1]


    return pi_copy

In [222]:
pol_imp(pi_speed, q_s_a, epsilon=0)

,n,s
0,1,0
10,1,0
20,1,0
30,1,0
40,1,0
50,1,0
60,1,0
70,1,0


### Policy iteration - MC Control

In [247]:
import time

num_ep = 10**3
beg_time =time.time()
q_s_a = q_s_a_init
pi=pi_50


for epi_i in range(1,num_ep) :
    sample_path_i = simul_path(pi, P_normal, P_speed, R_s_a)
    q_s_a = pol_eval_MC(sample_path_i, q_s_a, alpha = 1/epi_i)
    pi = pol_imp(pi, q_s_a, epsilon= 1/epi_i)
end_time =time.time()

print("Time difference of {} sec".format(end_time- beg_time))
print(pi.T)
print(q_s_a.T)

Time difference of 4.581876754760742 sec
     0   10   20   30   40   50   60   70
n  0.0  1.0  0.0  0.0  1.0  0.0  1.0  1.0
s  1.0  0.0  1.0  1.0  0.0  1.0  0.0  0.0
          0        10        20        30        40        50        60   70
n -5.590049 -3.903485 -3.562931 -2.830778 -1.706983 -1.699335 -0.927032  0.0
s -5.290829 -3.903570 -3.539539 -2.789214 -1.935418 -1.681056 -0.964113  0.0


In [234]:
import time

num_ep = 10**4
beg_time =time.time()
q_s_a = q_s_a_init
pi=pi_50


for epi_i in range(1,num_ep) :
    sample_path_i = simul_path(pi, P_normal, P_speed, R_s_a)
    q_s_a = pol_eval_MC(sample_path_i, q_s_a, alpha = 1/epi_i)
    pi = pol_imp(pi, q_s_a, epsilon= 1/epi_i)
end_time =time.time()

print("Time difference of {} sec".format(end_time- beg_time))
print(pi.T)
print(q_s_a.T)

Time difference of 47.17221975326538 sec
     0   10   20   30   40   50   60   70
n  0.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
          0        10        20        30        40        50        60   70
n -5.322128 -3.942366 -4.587156 -2.546016 -1.663601 -2.374797 -0.957560  0.0
s -5.127708 -3.942527 -3.455995 -3.856771 -1.663930 -1.663824 -1.522334  0.0


In [237]:
import time

num_ep = 10**5
beg_time =time.time()
q_s_a = q_s_a_init
pi=pi_50
exploration_rate = 1

for epi_i in range(1,num_ep) :
    sample_path_i = simul_path(pi, P_normal, P_speed, R_s_a)
    q_s_a = pol_eval_MC(sample_path_i, q_s_a, alpha = 1/epi_i)
    pi = pol_imp(pi, q_s_a, exploration_rate)
    exploration_rate*=0.9995 # exponential decay
    
end_time =time.time()

print("Time difference of {} sec".format(end_time- beg_time))
print(pi.T)
print(q_s_a.T)

Time difference of 467.4620771408081 sec
     0   10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0
s  1.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0
          0        10        20        30        40        50        60   70
n -5.931298 -4.895709 -4.229481 -2.818607 -1.930211 -2.267274 -1.000000  0.0
s -5.112680 -4.635681 -3.443544 -3.525068 -1.668082 -1.547125 -1.488869  0.0


### TD Control

In [239]:
import time

num_ep = 10**3
beg_time =time.time()
q_s_a = q_s_a_init
pi=pi_50


for epi_i in range(1,num_ep) :
    s_now="0"
    while s_now != "70":
        sample_step = simul_step(pi,s_now, P_normal, P_speed, R_s_a)
        q_s_a = pol_eval_TD(sample_step, q_s_a, alpha = 1/epi_i)
        pi = pol_imp(pi, q_s_a, epsilon= 1/epi_i)
        s_now = sample_step[3]
end_time =time.time()

print("Time difference of {} sec".format(end_time- beg_time))
print(pi.T)
print(q_s_a.T)

Time difference of 17.173409461975098 sec
     0   10   20   30   40   50   60   70
n  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
s  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
          0        10        20        30        40        50        60   70
n -3.266026 -2.753257 -2.469884 -2.125536 -1.387263 -1.560197 -0.980994  0.0
s -3.266162 -2.753411 -2.470245 -2.125874 -1.387263 -1.560875 -1.500000  0.0


In [240]:
import time

num_ep = 10**4
beg_time =time.time()
q_s_a = q_s_a_init
pi=pi_50


for epi_i in range(1,num_ep) :
    s_now="0"
    while s_now != "70":
        sample_step = simul_step(pi,s_now, P_normal, P_speed, R_s_a)
        q_s_a = pol_eval_TD(sample_step, q_s_a, alpha = 1/epi_i)
        pi = pol_imp(pi, q_s_a, epsilon= 1/epi_i)
        s_now = sample_step[3]
end_time =time.time()

print("Time difference of {} sec".format(end_time- beg_time))
print(pi.T)
print(q_s_a.T)

Time difference of 172.1427619457245 sec
     0   10   20   30   40   50   60   70
n  1.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0
s  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
          0        10        20        30       40        50        60   70
n -3.934944 -3.340103 -2.910932 -2.362509 -1.53358 -1.621797 -1.000000  0.0
s -3.934984 -3.340125 -2.910926 -2.362541 -1.53364 -1.621682 -1.009684  0.0


In [241]:
import time

num_ep = 10**3
beg_time =time.time()
q_s_a = q_s_a_init
pi=pi_50
exploration_rate = 1

for epi_i in range(1,num_ep) :
    s_now="0"
    while s_now != "70":
        sample_step = simul_step(pi,s_now, P_normal, P_speed, R_s_a)
        q_s_a = pol_eval_TD(sample_step, q_s_a, alpha = 1/epi_i)
        pi = pol_imp(pi, q_s_a, epsilon= 1/epi_i)
        s_now = sample_step[3]
        exploration_rate *=0.9995
end_time =time.time()

print("Time difference of {} sec".format(end_time- beg_time))
print(pi.T)
print(q_s_a.T)

Time difference of 17.474260330200195 sec
     0   10   20   30   40   50   60   70
n  0.0  0.0  1.0  1.0  0.0  1.0  1.0  1.0
s  1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0
          0        10        20        30        40        50        60   70
n -3.171516 -2.713767 -2.492095 -2.098770 -1.395104 -1.594247 -0.973156  0.0
s -3.171227 -2.713055 -2.492301 -2.099338 -1.395038 -1.594936 -1.500000  0.0


In [242]:
pi

,n,s
0,0.0,1.0
10,0.0,1.0
20,1.0,0.0
30,1.0,0.0
40,0.0,1.0
50,1.0,0.0
60,1.0,0.0
70,1.0,0.0
